In [15]:
from google.colab import drive
import os

# Perintah ini akan memunculkan jendela pop-up untuk meminta izin
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/Dataset-Skripsi'
dataset_dir = os.path.join(base_dir, 'Dataset')
output_dir = os.path.join(base_dir, 'Output')

# Membuat folder jika belum ada
os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

print("Google Drive terhubung dan folder siap.")
print(f"Folder Dataset: {dataset_dir}")
print(f"Folder Output: {output_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive terhubung dan folder siap.
Folder Dataset: /content/drive/MyDrive/Dataset-Skripsi/Dataset
Folder Output: /content/drive/MyDrive/Dataset-Skripsi/Output


In [16]:
# Install MTCNN untuk deteksi wajah
!pip install mtcnn

In [17]:
# Untuk Deep Learning (TensorFlow & Keras)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Untuk Pengolahan Angka dan Gambar
import numpy as np
import cv2  # Ini adalah OpenCV
import matplotlib.pyplot as plt

# Untuk Utilitas Sistem dan File
import os
import glob
import random

# Untuk Deteksi Wajah (yang baru kita install)
from mtcnn.mtcnn import MTCNN

# Untuk Evaluasi Model
from sklearn.metrics import classification_report, confusion_matrix

print("Semua library dasar berhasil di-impor.")

Semua library dasar berhasil di-impor.


In [18]:
# Tentukan path output Anda (sudah didefinisikan di sel 1, tapi kita definisikan lagi di sini untuk jelas)
output_dir = '/content/drive/MyDrive/Dataset-Skripsi/Output'

# Tentukan path untuk train, validation, dan test
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'validation')
test_dir = os.path.join(output_dir, 'test')

print("Struktur folder train/validation/test telah dibuat di dalam Output.")

Struktur folder train/validation/test telah dibuat di dalam Output.


In [19]:
# Inisialisasi detektor MTCNN
detector = MTCNN()
print("Detektor MTCNN siap digunakan.")

Detektor MTCNN siap digunakan.


In [20]:
import cv2
import os
from mtcnn.mtcnn import MTCNN
import glob

# Inisialisasi detektor lagi (untuk memastikan, jika sel terpisah)
try:
    detector
except NameError:
    detector = MTCNN()

# Path dari sel 1
dataset_dir = '/content/drive/MyDrive/Dataset-Skripsi/Dataset'
output_dir = '/content/drive/MyDrive/Dataset-Skripsi/Output'

# Tentukan berapa frame yang ingin Anda ambil per video
# Mengambil 20 frame akan memberi Anda 20 gambar wajah per video.
# Sesuaikan angka ini berdasarkan ukuran dataset Anda.
FRAMES_PER_VIDEO = 20

def extract_faces_from_video(video_path, output_folder, frames_to_extract):
    """Membaca video, mengekstrak N frame, mendeteksi wajah, dan menyimpannya."""
    video_capture = cv2.VideoCapture(video_path)
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

    # Jika video tidak bisa dibuka
    if not video_capture.isOpened():
        print(f"Error: Tidak bisa membuka video {video_path}")
        return

    # Hitung interval frame yang akan diambil
    if total_frames <= 0:
        print(f"Error: Video {video_path} tidak memiliki frame.")
        return

    frame_interval = max(1, total_frames // frames_to_extract)

    frame_count = 0
    saved_face_count = 0

    while frame_count < total_frames and saved_face_count < frames_to_extract:
        # Set video ke frame yang diinginkan
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
        success, frame = video_capture.read()

        if not success:
            frame_count += frame_interval # Lompat ke frame selanjutnya jika gagal baca
            continue

        # Konversi frame BGR (OpenCV) ke RGB (MTCNN)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Deteksi wajah
        faces = detector.detect_faces(frame_rgb)

        if faces:
            # Ambil wajah pertama (biasanya yang paling besar/paling jelas)
            face_data = faces[0]
            x, y, w, h = face_data['box']

            # Pastikan koordinat tidak negatif (beberapa bug di MTCNN)
            x, y = max(0, x), max(0, y)

            # Potong (crop) wajah dari frame ASLI (BGR)
            face_image = frame[y:y+h, x:x+w]

            # Simpan gambar wajah
            # Resize wajah ke ukuran standar, misal 224x224 (untuk model CNN nanti)
            face_image_resized = cv2.resize(face_image, (224, 224))

            # Buat nama file yang unik
            video_name = os.path.basename(video_path).split('.')[0]
            output_filename = os.path.join(output_folder, f"{video_name}_frame_{frame_count}_face_{saved_face_count}.jpg")

            cv2.imwrite(output_filename, face_image_resized)
            saved_face_count += 1

        # Lompat ke interval frame selanjutnya
        frame_count += frame_interval

    video_capture.release()
    # print(f"Selesai memproses {video_path}. Total wajah disimpan: {saved_face_count}")

print("Fungsi extract_faces_from_video() telah didefinisikan.")

Fungsi extract_faces_from_video() telah didefinisikan.


In [21]:
import random
import os # Pastikan os sudah diimpor, yang sudah ada di sel awal

# Tentukan rasio pembagian data
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# Path folder output (dari sel 4)
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'validation')
test_dir = os.path.join(output_dir, 'test')

# Path folder input (Dataset)
real_videos_path = os.path.join(dataset_dir, 'Celeb-real')
fake_videos_path = os.path.join(dataset_dir, 'Celeb-synthesis')

# Loop untuk REAL dan FAKE
for (video_type, source_path) in [("Celeb-real", real_videos_path), ("Celeb-synthesis", fake_videos_path)]:
    print(f"\nMemulai memproses video kategori: {video_type}...")

    # Ambil semua file video (misal .mp4)
    video_files = glob.glob(os.path.join(source_path, "*.mp4"))

    if not video_files:
        print(f"PERINGATAN: Tidak ada file .mp4 ditemukan di {source_path}")
        continue

    random.shuffle(video_files) # Acak urutan video

    # Hitung jumlah data untuk tiap set
    total_videos = len(video_files)
    train_count = int(total_videos * TRAIN_RATIO)
    val_count = int(total_videos * VAL_RATIO)

    # Bagi daftar video
    train_videos = video_files[:train_count]
    val_videos = video_files[train_count : train_count + val_count]
    test_videos = video_files[train_count + val_count:]

    print(f"Total video {video_type}: {total_videos}")
    print(f"Data latih (train): {len(train_videos)} video")
    print(f"Data validasi (val): {len(val_videos)} video")
    print(f"Data uji (test): {len(test_videos)} video")

    # Proses dan simpan ke folder yang sesuai
    # 1. Proses TRAIN
    print(f"Memproses {video_type} untuk TRAIN...")
    output_folder_train = os.path.join(train_dir, video_type)
    os.makedirs(output_folder_train, exist_ok=True) # Tambahkan ini untuk membuat folder kelas
    for video_file in train_videos:
        extract_faces_from_video(video_file, output_folder_train, FRAMES_PER_VIDEO)

    # 2. Proses VALIDATION
    print(f"Memproses {video_type} untuk VALIDATION...")
    output_folder_val = os.path.join(val_dir, video_type)
    os.makedirs(output_folder_val, exist_ok=True) # Tambahkan ini untuk membuat folder kelas
    for video_file in val_videos:
        extract_faces_from_video(video_file, output_folder_val, FRAMES_PER_VIDEO)

    # 3. Proses TEST
    print(f"Memproses {video_type} untuk TEST...")
    output_folder_test = os.path.join(test_dir, video_type)
    os.makedirs(output_folder_test, exist_ok=True) # Tambahkan ini untuk membuat folder kelas
    for video_file in test_videos:
        extract_faces_from_video(video_file, output_folder_test, FRAMES_PER_VIDEO)

print("\n--- PROSES EKSTRAKSI SELESAI ---")


Memulai memproses video kategori: Celeb-real...
Total video Celeb-real: 309
Data latih (train): 216 video
Data validasi (val): 46 video
Data uji (test): 47 video
Memproses Celeb-real untuk TRAIN...


KeyboardInterrupt: 

In [ ]:
# Tentukan path dari Fase 1
output_dir = '/content/drive/MyDrive/Dataset-Skripsi/Output'
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'validation')
test_dir = os.path.join(output_dir, 'test')

# Tentukan parameter gambar
# Ukuran ini HARUS SAMA dengan yang Anda gunakan di Fase 1 (224x224)
IMG_SIZE = 224
BATCH_SIZE = 32 # Berapa gambar yang dimuat sekaligus

# 1. Generator untuk Data Latih (dengan Augmentasi)
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Normalisasi piksel dari 0-255 ke 0-1
    rotation_range=20,          # Putar gambar secara acak
    width_shift_range=0.2,      # Geser horizontal
    height_shift_range=0.2,     # Geser vertikal
    shear_range=0.2,            # Miringkan gambar
    zoom_range=0.2,             # Zoom gambar
    horizontal_flip=True,       # Balik gambar secara horizontal
    fill_mode='nearest'
)

# 2. Generator untuk Data Validasi & Tes (TANPA Augmentasi, hanya rescale)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# 3. Buat 'Pipa' yang membaca dari folder
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary' # Karena kita punya 2 kelas: REAL vs FAKE
)

validation_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False # Tidak perlu diacak untuk validasi
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False # Tidak perlu diacak untuk tes
)

print(f"Kelas ditemukan: {train_generator.class_indices}")

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

# Tentukan ukuran input
input_shape = (IMG_SIZE, IMG_SIZE, 3)

# 1. Muat Model Dasar (XceptionNet)
# include_top=False berarti kita tidak menyertakan lapisan klasifikasi aslinya
# weights='imagenet' berarti kita memuat bobot yang sudah terlatih
base_model_xception = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

# 2. Bekukan lapisan dasar
# Kita tidak ingin melatih ulang lapisan yang sudah pintar ini (untuk saat ini)
base_model_xception.trainable = False

# 3. Tambahkan 'Kepala' (Head) baru kita di atas model dasar
x = base_model_xception.output
x = GlobalAveragePooling2D()(x) # Meratakan output fitur
x = Dense(1024, activation='relu')(x) # Lapisan tersembunyi (hidden layer)
x = Dropout(0.5)(x) # Mencegah overfitting
# Lapisan output: 1 neuron dengan aktivasi sigmoid (menghasilkan angka 0-1)
predictions = Dense(1, activation='sigmoid')(x)

# 4. Gabungkan menjadi model akhir
model_xception = Model(inputs=base_model_xception.input, outputs=predictions)

# 5. Kompilasi Model
# Kita gunakan 'binary_crossentropy' karena ini masalah 2 kelas (REAL vs FAKE)
model_xception.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # Gunakan learning rate kecil
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Tampilkan arsitektur model
model_xception.summary()

In [ ]:
# Tentukan berapa epoch (putaran) Anda ingin melatih
EPOCHS = 5 # Mulai dengan 10, Anda bisa tambah nanti

# Hitung jumlah langkah per epoch
# Ini penting agar Keras tahu berapa banyak batch yang harus diambil
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = validation_generator.samples // BATCH_SIZE

print(f"Total langkah per epoch (train): {steps_per_epoch}")
print(f"Total langkah validasi: {validation_steps}")

# Mulai pelatihan!
history_xception = model_xception.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

print("\n--- PELATIHAN XCEPTIONNET SELESAI ---")

In [ ]:
# Simpan model yang sudah dilatih
model_path_xception = os.path.join(output_dir, 'model_xception_final.h5')
model_xception.save(model_path_xception)
print(f"Model XceptionNet disimpan di: {model_path_xception}")

# Simpan riwayat pelatihan (untuk dibuat grafik nanti)
import pickle # Pickle digunakan untuk menyimpan objek Python

history_path_xception = os.path.join(output_dir, 'history_xception.pkl')
with open(history_path_xception, 'wb') as f:
    pickle.dump(history_xception.history, f)

print(f"Riwayat pelatihan disimpan di: {history_path_xception}")

# --- (OPSIONAL) Plot Grafik Pelatihan ---
plt.figure(figsize=(12, 4))

# Plot Akurasi
plt.subplot(1, 2, 1)
plt.plot(history_xception.history['accuracy'], label='Training Accuracy')
plt.plot(history_xception.history['val_accuracy'], label='Validation Accuracy')
plt.title('Akurasi Model XceptionNet')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history_xception.history['loss'], label='Training Loss')
plt.plot(history_xception.history['val_loss'], label='Validation Loss')
plt.title('Loss Model XceptionNet')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()